In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
batch_size = 128
num_hidden_neurons = 1024
num_hidden_neurons2 = 350

#source:  http://arxiv.org/pdf/1502.01852v1.pdf from https://github.com/jantic/UdacityDeepLearningProjects/blob/master/3%20-%20Regularization/Main.py
def calculateOptimalWeightStdDev(numPreviousLayerParams):
    return math.sqrt(2.0/numPreviousLayerParams)

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)


    # 1 - HIDDEN LAYER 
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_hidden_neurons], stddev=calculateOptimalWeightStdDev(image_size * image_size)))
    hidden_biases = tf.Variable(tf.zeros([num_hidden_neurons]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    # hidden_layer = tf.nn.dropout(hidden_layer, 0.5)


    # 2 - HIDDEN LAYER 
    hidden_weights2 = tf.Variable(tf.truncated_normal([num_hidden_neurons, num_hidden_neurons2], stddev=calculateOptimalWeightStdDev(num_hidden_neurons)))
    hidden_biases2 = tf.Variable(tf.zeros([num_hidden_neurons2]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer, hidden_weights2) + hidden_biases2)
    # hidden_layer = tf.nn.dropout(hidden_layer, 0.5)


    # OUT LAYER
    
    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([num_hidden_neurons2, num_labels], stddev=calculateOptimalWeightStdDev(num_hidden_neurons)))
    biases = tf.Variable(tf.zeros([num_labels]))
    layer = tf.matmul(hidden_layer2, weights) + biases

    # Training computation.    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(layer, tf_train_labels))
    beta = 5e-4 # value chosen from copying line 237 of this file https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/image/mnist/convolutional.py
    L2_regul = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) + tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    loss = loss + beta * L2_regul

    # Optimizer.
    # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Conforme avanzamos en el entrenamiento (cada 500 steps) hacemos descender (a un 90%) la tasa de aprendizaje.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 500, 0.95)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(layer)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases),hidden_weights2)+hidden_biases2),weights)+biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases),hidden_weights2)+hidden_biases2),weights)+biases)

In [5]:
num_steps = 3001

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # tf.global_variables_initializer().run()
    tf.initialize_all_variables().run()
    print("Initialized")

    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))

        global_step += 1
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.734388
Minibatch accuracy: 7.8%
Validation accuracy: 31.4%
Minibatch loss at step 500: 0.831214
Minibatch accuracy: 82.0%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.577770
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 1500: 0.549615
Minibatch accuracy: 89.8%
Validation accuracy: 88.2%
Minibatch loss at step 2000: 0.563788
Minibatch accuracy: 89.8%
Validation accuracy: 88.8%
Minibatch loss at step 2500: 0.614593
Minibatch accuracy: 88.3%
Validation accuracy: 88.5%
Minibatch loss at step 3000: 0.393197
Minibatch accuracy: 92.2%
Validation accuracy: 89.8%
Test accuracy: 94.8%
